In [1]:
# ── data models -------------------------------------------------------------
from pydantic import BaseModel, Field
from typing import List, Literal

# ── agent -------------------------------------------------------------------
from pydantic_ai.providers.openai import OpenAIProvider
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai import Agent

In [2]:
class Issue(BaseModel):
    line: int = Field(..., description="line number (1-based)")
    severity: Literal["info", "warning", "error"]
    message: str
    suggestion: str

class ReviewResult(BaseModel):
    summary: str
    issues: List[Issue]

In [3]:
# provider that talks to llama-cpp
llama = OpenAIProvider(
    base_url="http://llama_cpp:8000/v1",
    api_key="local",
)

model = OpenAIModel(
    "Nous-Hermes-2-DPO_into_Nous_Hermes-2-Pro.Q8_0.gguf", 
    provider=llama,
)

reviewer = Agent(
    model,
    output_type=ReviewResult,
    instructions=(
        "You are a senior Python engineer. "
        "Act as a strict code reviewer:\n"
        "1. Read the code the user sends.\n"
        "2. List all PEP-8, type, logic and functional problems.\n"
        "3. Suggest concrete improvements.\n"
        "Return **only** JSON that matches the ReviewResult schema."
    ),
    # model_settings={
    #     "temperature": 0.6,   # more deterministic than the default 0.7
    #     "top_p": 0.95,        # nucleus sampling – don’t set if you’re using temperature
    #     "max_tokens": 2048,    # example of another setting you can pass
    # },
)

In [ ]:
# ── review some code --------------------------------------------------------
code = """
def ADD(a: int,b: str):
  return a -  b
"""

result = await reviewer.run(code)      # top-level await works in Jupyter ≥7

In [ ]:
print(result.output.model_dump_json(indent=2))